In [23]:
from PIL import Image
import torchvision.transforms as transforms 
from src.metrics.PSNR import PSNR
from diffusers import LDMSuperResolutionPipeline
import torch

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "CompVis/ldm-super-resolution-4x-openimages"

In [3]:
# load model and scheduler
pipeline = LDMSuperResolutionPipeline.from_pretrained(model_id)
pipeline = pipeline.to(device)

unet/diffusion_pytorch_model.safetensors not found
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# let's download an  image
url = "/home/arazin/main/work/HUAWEI/ISR/ISR_With_Depth_Estimation/ldm_custom_pipeline/datasets/images/testset/85135_z98_RussiaSaint-Petersburg.jpg"
img = Image.open(url).convert("RGB")
low_res_img = img.resize((128, 128))

# run pipeline in inference (sample random noise and denoise)
upscaled_image = pipeline(low_res_img, num_inference_steps=100, eta=1).images[0]
# save image
low_res_img.save("low_res_2.png")
upscaled_image.save("ldm_generated_image_2.png")

  0%|          | 0/100 [00:00<?, ?it/s]

In [22]:
true = img.resize(upscaled_image.size)
true.size, upscaled_image.size

((512, 512), (512, 512))

In [20]:
transform = transforms.Compose([ 
    transforms.PILToTensor() 
]) 

img_tensor_true = transform(true) / 255
img_tensor_prediction = transform(upscaled_image) / 255

print(img_tensor_true.shape, img_tensor_prediction.shape)

torch.Size([3, 512, 512]) torch.Size([3, 512, 512])


In [21]:
PSNR(img_tensor_true, img_tensor_prediction)

26.33388328552246